# Adaptive Computational and Machine Learning
## Stock Prediction Using Unsupervised Learning

### Team Members
1.
2.
3.

# Python Imports

## Python Installations

In [3]:
# !pip install PyPortfolio
# !pip install yfinance
# !pip install pandas_datareader
# !pip install pandas_ta
# !pip install setuptools
# !pip install pandas_datareader

## Python Libraries

In [4]:
from statsmodels.regression.rolling import RollingOLS
# import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

# Data Exploratory Analysis

## Downloading Data

In [5]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [6]:
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()
symbols_list

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CHRW',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA'

In [7]:
end_date = '2024-05-22'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*4)
print(f'Download SP500 prices from {start_date} to {end_date}: \n')

df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack()
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()
df

Download SP500 prices from 2020-05-23 00:00:00 to 2024-05-22: 



[*********************100%%**********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2020-05-26 A        83.889847   86.129997   86.790001   85.639999   86.230003   
           AAL      11.140000   11.140000   11.280000   10.250000   10.270000   
           AAPL     77.330833   79.182503   81.059998   79.125000   80.875000   
           ABBV     76.594925   90.709999   93.570000   90.309998   93.449997   
           ABT      83.534447   89.480003   93.290001   89.169998   92.949997   
...                       ...         ...         ...         ...         ...   
2024-05-21 XYL     145.949997  145.949997  146.080002  144.380005  145.559998   
           YUM     139.300003  139.300003  140.039993  138.539993  140.020004   
           ZBH     119.000000  119.000000  119.669998  118.949997  119.379997   
           ZBRA    321.779999  321.779999  322.480011  317.070007  319.209991   
           ZTS     172.789993  172.789993  175.779999  172.520004  173.470001   

Price                   volume  
date       ticker               
2020-05-26 A         3173400.0  
           AAL     101717000.0  
           AAPL    125522000.0  
           ABBV     11869800.0  
           ABT       6243800.0  
...                        ...  
2024-05-21 XYL        961300.0  
           YUM       1192000.0  
           ZBH        776600.0  
           ZBRA       285700.0  
           ZTS       1600400.0  

[500790 rows x 6 columns]

# Data Preprocessing

## Feature Engineering

In [8]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

In [9]:
def indicator(data, new_column_name, indicator_type, new_column_int_location=None):
    def calculate_indicator(series):
        if indicator_type == 'rsi':
            return pandas_ta.rsi(close=series, length=20)
        elif indicator_type == 'bbands':
            bbands = pandas_ta.bbands(close=np.log1p(series), length=20)
            return bbands.iloc[:, new_column_int_location]
        else:
            raise ValueError("Unsupported indicator type")

    grouped = data.groupby(level=1)['adj close'].apply(calculate_indicator)
    data[new_column_name] = grouped.reset_index(level=0, drop=True)

    return data

def compute_atr(data):
    atr = pandas_ta.atr(high=data['high'],
                        low=data['low'],
                        close=data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

In [10]:
df = indicator(df, 'rsi', 'rsi')
df = indicator(df, 'bb_low', 'bbands', 0)
df = indicator(df, 'bb_mid', 'bbands', 1)
df = indicator(df, 'bb_high', 'bbands', 2)
df['dollar_volume'] = (df['adj close']*df['volume'])/1e6
df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

In [11]:
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2020-05-26 A        83.889847   86.129997   86.790001   85.639999   86.230003   
           AAL      11.140000   11.140000   11.280000   10.250000   10.270000   
           AAPL     77.330833   79.182503   81.059998   79.125000   80.875000   
           ABBV     76.594925   90.709999   93.570000   90.309998   93.449997   
           ABT      83.534447   89.480003   93.290001   89.169998   92.949997   
...                       ...         ...         ...         ...         ...   
2024-05-21 XYL     145.949997  145.949997  146.080002  144.380005  145.559998   
           YUM     139.300003  139.300003  140.039993  138.539993  140.020004   
           ZBH     119.000000  119.000000  119.669998  118.949997  119.379997   
           ZBRA    321.779999  321.779999  322.480011  317.070007  319.209991   
           ZTS     172.789993  172.789993  175.779999  172.520004  173.470001   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2020-05-26 A         3173400.0         -0.000203        NaN       NaN   
           AAL     101717000.0          0.002030        NaN       NaN   
           AAPL    125522000.0         -0.000484        NaN       NaN   
           ABBV     11869800.0         -0.014653        NaN       NaN   
           ABT       6243800.0         -0.003386        NaN       NaN   
...                        ...               ...        ...       ...   
2024-05-21 XYL        961300.0          0.000066  74.780260  4.861243   
           YUM       1192000.0          0.000048  52.915312  4.900955   
           ZBH        776600.0          0.000014  42.335440  4.782875   
           ZBRA       285700.0          0.000118  62.394761  5.670350   
           ZTS       1600400.0          0.000169  57.165071  5.033590   

Price                bb_mid   bb_high  dollar_volume      macd  
date       ticker                                               
2020-05-26 A            NaN       NaN     266.216040       NaN  
           AAL          NaN       NaN    1133.127415       NaN  
           AAPL         NaN       NaN    9706.720874       NaN  
           ABBV         NaN       NaN     909.166440       NaN  
           ABT          NaN       NaN     521.572378       NaN  
...                     ...       ...            ...       ...  
2024-05-21 XYL     4.938055  5.014867     140.301732  1.729426  
           YUM     4.939115  4.977274     166.045604  0.063408  
           ZBH     4.798865  4.814856      92.415400 -0.486613  
           ZBRA    5.745455  5.820560      91.932546  0.760251  
           ZTS     5.115926  5.198263     276.533105  0.961701  

[500790 rows x 13 columns]

## Aggregate to monthly level and filter top 150 most liquid stocks for each month.

-  To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.

In [13]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2020-06-30 A          146.735282   86.252312          0.000114  53.696910   
           AAL       2248.422414   13.070000          0.001035  47.609792   
           AAPL     12525.301465   89.067291          0.000092  66.610040   
           ABBV       682.057171   82.902542         -0.007785  63.521535   
           ABT        543.091494   85.354874          0.000149  53.239488   
...                          ...         ...               ...        ...   
2024-05-31 XYL        192.058552  145.949997          0.000066  74.780260   
           YUM        289.226311  139.300003          0.000048  52.915312   
           ZBH        165.083659  119.000000          0.000014  42.335440   
           ZBRA       102.243351  321.779999          0.000118  62.394761   
           ZTS        466.645575  172.789993          0.000169  57.165071   

                     bb_low    bb_mid   bb_high      macd  
date       ticker                                          
2020-06-30 A       4.428306  4.465683  4.503059 -0.364183  
           AAL     2.521250  2.797671  3.074091 -1.077455  
           AAPL    4.378892  4.453614  4.528337  0.983870  
           ABBV    4.358660  4.399743  4.440826  0.452446  
           ABT     4.412652  4.442859  4.473066 -0.445453  
...                     ...       ...       ...       ...  
2024-05-31 XYL     4.861243  4.938055  5.014867  1.729426  
           YUM     4.900955  4.939115  4.977274  0.063408  
           ZBH     4.782875  4.798865  4.814856 -0.486613  
           ZBRA    5.670350  5.745455  5.820560  0.760251  
           ZTS     5.033590  5.115926  5.198263  0.961701  

[23915 rows x 8 columns]